# Scrapping le site web 

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time


In [14]:
import requests
from bs4 import BeautifulSoup

In [44]:
driver = webdriver.Chrome()

driver.get('https://www.cuisinez-pour-bebe.fr/recette-bebe/')

wait = WebDriverWait(driver, 200)  # 10 secondes d'attente
wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'post-content')))

last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Faire défiler jusqu'à la fin de la page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Attendre le chargement de la nouvelle partie de la page
    time.sleep(5)  # Attendez quelques secondes pour que les nouveaux éléments se chargent

    # Calculer la nouvelle hauteur de défilement et comparer avec la dernière hauteur de défilement
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# Maintenant que la page est entièrement chargée, récupérez les liens
post_content = driver.find_element(By.CLASS_NAME, 'post-content')
hits_container = post_content.find_element(By.ID, 'hits')
links = [a.get_attribute('href') for a in hits_container.find_elements(By.TAG_NAME, 'a')]

print(links)

driver.quit()


['https://www.cuisinez-pour-bebe.fr/ma-premiere-puree-de-carotte/', 'https://www.cuisinez-pour-bebe.fr/ma-premiere-puree-de-courgette/', 'https://www.cuisinez-pour-bebe.fr/pancakes-a-la-banane-sans-oeufs/', 'https://www.cuisinez-pour-bebe.fr/muffins-pomme-amandes/', 'https://www.cuisinez-pour-bebe.fr/ma-premiere-puree-de-betterave/', 'https://www.cuisinez-pour-bebe.fr/ma-premiere-puree-de-courge-butternut/', 'https://www.cuisinez-pour-bebe.fr/rostis-pommes-de-terre-courgettes/', 'https://www.cuisinez-pour-bebe.fr/cookies-moelleux-carottes-petits-pois/', 'https://www.cuisinez-pour-bebe.fr/pancakes-a-la-betterave-et-sarrasin/', 'https://www.cuisinez-pour-bebe.fr/creme-de-mais-bebe/', 'https://www.cuisinez-pour-bebe.fr/duo-de-petits-pois-et-carottes-au-jambon/', 'https://www.cuisinez-pour-bebe.fr/parmentier-de-panais-au-canard-2/', 'https://www.cuisinez-pour-bebe.fr/puree-fine-de-lentilles-corail-potimarron-et-noisettes/', 'https://www.cuisinez-pour-bebe.fr/taboule-parfume/', 'https://www

In [47]:
len(links)

137

In [49]:
def scrape_recipe_details(recipe_url,fin_links):
    response = requests.get(recipe_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        

        # Vérification de la classe 'ihc-locker-wrap'
        if soup.find('div', class_='ihc-locker-wrap'):
            # Retourne une valeur spécifique ou passe simplement
            print("Locked content, skipping...")
            return -1

        else:

            # ingredient
            ingredients_elements = soup.find_all('li', class_='wprm-recipe-ingredient')
            ingredients = [ingredient.get_text().strip() for ingredient in ingredients_elements]

            # image
            image_container = soup.find('div', class_='wprm-recipe-image')
            image_element = image_container.find('img') if image_container else None
            image_url = image_element['src'] if image_element and 'src' in image_element.attrs else None

            # age
            age_element = soup.find('div', class_='post-categories').find('a')
            recommended_age = age_element.get_text().strip() if age_element else None

            # materiel
            materiel_element = soup.find_all('li', class_='wprm-recipe-equipment-item')
            materiels = [materiel.get_text().strip() for materiel in materiel_element]

            # tag
            tag_elements = soup.find('div', class_='post-tags').find_all('a')
            tags = [tag.get_text().strip() for tag in tag_elements]

            # titre
            title_element = soup.find('h1', class_='post-title')
            title = title_element.get_text().strip() if title_element else None

            fin_links.append(recipe_url)

            return ingredients, image_url, recommended_age, materiels, tags, title, fin_links
        
    else:
        
        return None, None, None, None, None, None, None

In [58]:
fin_links=[]
recettes = []

for link in links:
    result = scrape_recipe_details(link,fin_links)
    if(result!=-1):
        ingredients, image_url, recommended_age, materiel, tags, title, fin_links = result
        if ingredients and title:
            print(f"Recette pour : {title}")
            print(f"Âge recommandé: {recommended_age}")
            print(f"Image URL: {image_url}")
            print(f"Matériel: {materiel}")
            print(f"Ingrédients pour {link} : {ingredients}")
            print(f"Tags: {tags}\n")
            recette = {
                "titre": title,
                "âge_recommandé": recommended_age,
                "url_image": image_url,
                "matériel": materiel,
                "ingrédients": ingredients,
                "tags": tags
            }
            recettes.append(recette)
            
    else:
        print(f"Impossible de récupérer les détails pour {link}")

# Écriture dans un fichier JSON
with open('recettes.json', 'w') as json_file:
    json.dump(recettes, json_file, ensure_ascii=False, indent=4)

print("Fichier JSON des recettes créé avec succès.")

Recette pour : Ma première purée de carotte
Âge recommandé: dès 4 mois
Image URL: https://cdn.cuisinez-pour-bebe.fr/wp-content/uploads/2019/09/premiere-puree-carotte-bebe-diversification-4-mois.jpg
Matériel: ['▢ cuit-vapeur (ou panier vapeur)', '▢ mixeur']
Ingrédients pour https://www.cuisinez-pour-bebe.fr/ma-premiere-puree-de-carotte/ : ['▢ 200 g carotte(s) (bio)', "▢ eau (convenant à l'alimentation du nourrisson)"]
Tags: ['premiers repas', 'sans gluten', 'sans oeuf', 'sans PLV', 'vegan', 'végétarien', 'automne', 'été', 'hiver', 'printemps', 'plat du midi', 'plat du soir', 'Recettes contre la diarrhée / gastro 🤮']

Recette pour : Ma première purée de courgette
Âge recommandé: dès 4 mois
Image URL: https://cdn.cuisinez-pour-bebe.fr/wp-content/uploads/2019/09/premiere-puree-courgette-bebe-diversification-4-mois-6-mois.jpg
Matériel: ['▢ cuit-vapeur (ou panier vapeur)', '▢ mixeur']
Ingrédients pour https://www.cuisinez-pour-bebe.fr/ma-premiere-puree-de-courgette/ : ['▢ 300 g courgette(s)'

In [51]:
len(fin_links)

270

In [53]:
fin_links

['https://www.cuisinez-pour-bebe.fr/ma-premiere-puree-de-carotte/',
 'https://www.cuisinez-pour-bebe.fr/ma-premiere-puree-de-carotte/',
 'https://www.cuisinez-pour-bebe.fr/ma-premiere-puree-de-courgette/',
 'https://www.cuisinez-pour-bebe.fr/ma-premiere-puree-de-courgette/',
 'https://www.cuisinez-pour-bebe.fr/pancakes-a-la-banane-sans-oeufs/',
 'https://www.cuisinez-pour-bebe.fr/pancakes-a-la-banane-sans-oeufs/',
 'https://www.cuisinez-pour-bebe.fr/muffins-pomme-amandes/',
 'https://www.cuisinez-pour-bebe.fr/muffins-pomme-amandes/',
 'https://www.cuisinez-pour-bebe.fr/ma-premiere-puree-de-betterave/',
 'https://www.cuisinez-pour-bebe.fr/ma-premiere-puree-de-betterave/',
 'https://www.cuisinez-pour-bebe.fr/ma-premiere-puree-de-courge-butternut/',
 'https://www.cuisinez-pour-bebe.fr/ma-premiere-puree-de-courge-butternut/',
 'https://www.cuisinez-pour-bebe.fr/rostis-pommes-de-terre-courgettes/',
 'https://www.cuisinez-pour-bebe.fr/rostis-pommes-de-terre-courgettes/',
 'https://www.cuisin

In [54]:
import numpy as np

In [55]:
fin_links_unique = np.unique(fin_links)

In [56]:
len(fin_links_unique)

135

In [52]:
len(links)

137